In [5]:
import pandas as pd
import altair as alt

from IPython.display import Markdown

_ = alt.renderers.set_embed_options(theme="ggplot2", actions=False)


In [ ]:
%load_ext sql

In [2]:
%%sql daily_data_on_chain <<
select
    date_trunc('day', sector_start_at) as date,
    sum(piece_size / 1024 ** 5) as onboarded_tibs
from filecoin_state_market_deals
where 
    sector_start_epoch > 0 and slash_epoch = -1
    and sector_start_at < current_date
group by 1
order by 1


In [6]:
df = daily_data_on_chain.DataFrame()
df["date"] = pd.to_datetime(df["date"])


In [8]:
# Altair chart of daily data on chain

c = (
    alt.Chart(df)
    .mark_area()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("onboarded_tibs:Q", title="Onboarded TiBs"),
        color=alt.value("#55595c"),
        tooltip=["date:T", "onboarded_tibs:Q"],
    )
    .properties(width="container", height=400, title="Data Onboarding Rate")
)

# Make it zoomable and lock the y axis
c = c.interactive(bind_y=False)

c.display()


alt.Chart(...)

In [9]:
# Now cumulative data on chain

df["cumulative_data"] = df["onboarded_tibs"].cumsum()

alt.Chart(df).mark_area().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("cumulative_data:Q", title="Cumulative data on chain (PiB)"),
    color=alt.value("#55595c"),
    tooltip=["date:T", "cumulative_data:Q"],
).properties(
    width="container",
    height=400,
    title="Cumulative data on chain",
)


alt.Chart(...)

In [10]:
# Gwoth rate of onboarded_tibs (7 day rolling average)

df["growth_rate"] = df["onboarded_tibs"].pct_change(periods=7).rolling(7).mean() * 100
recent_df = df[df["date"] > "2023-01-01"]

alt.Chart(recent_df).mark_bar().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("growth_rate:Q", title="Growth rate"),
    color=alt.value("#55595c"),
    tooltip=["date:T", "growth_rate:Q"],
).properties(
    width="container",
    height=400,
    title="Growth rate of onboarded data (7 day rolling average) in 2023",
)


alt.Chart(...)

In [17]:
df["delta"] = df["onboarded_tibs"].diff()

# Color negative values red
alt.Chart(df).mark_bar().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("delta:Q", title="Daily delta"),
    color=alt.condition(
        alt.datum.delta > 0,
        alt.value("#4e6357"),  # The positive color
        alt.value("#634e50"),  # The negative color
    ),
    tooltip=["date:T", "delta:Q"],
).properties(
    width="container",
    height=400,
    title="Daily delta of onboarded data",
).interactive(
    bind_y=False
)


alt.Chart(...)